In [1]:
import sys
sys.path.append('../')

In [2]:
import os
import tensorflow as tf
import tensorflow_addons as tfa

def conv_block(inputs, filters, kernel_size, strides, padding, use_bias):
    x = tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, 
                                padding = padding, use_bias = use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def conv_block_1(inputs, filters):
    x1 = conv_block(inputs, filters=filters, kernel_size=1, strides=1, padding='same', use_bias=True)
    x1 = conv_block(x1, filters=filters, kernel_size=1, strides=1, padding='same', use_bias=True)
    return x1

def conv_block_3(inputs, filters):
    x3 = conv_block(inputs, filters=filters, kernel_size=3, strides=1, padding='same', use_bias=True)
    x3 = conv_block(x3, filters=filters, kernel_size=3, strides=1, padding='same', use_bias=True)
    return x3

def conv_block_5(inputs, filters):
    x5 = conv_block(inputs, filters=filters, kernel_size=5, strides=1, padding='same', use_bias=True)
    x5 = conv_block(x5, filters=filters, kernel_size=5, strides=1, padding='same', use_bias=True)
    return x5

def conv_3_1(inputs, filters):
    x1 = conv_block_1(inputs, filters)
    x3 = conv_block_3(inputs, filters)
    x5 = conv_block_5(inputs, filters)
    x = tf.keras.layers.concatenate(inputs=[x1, x3, x5], axis=-1)
    x = tf.keras.layers.Conv2D(filters = filters, kernel_size = 1, strides = 1, 
                                padding = 'same', use_bias = True)(x)
    return x

def conv_3_1_1(inputs, filters):
    x1 = conv_block_1(inputs, filters)
    x3 = conv_block_3(inputs, filters)
    x5 = conv_block_5(inputs, filters)
    x = tf.keras.layers.concatenate(inputs=[x1, x3, x5], axis=-1)
    x = tf.keras.layers.Conv2D(filters = filters, kernel_size = 1, strides = 1, 
                                padding = 'same', use_bias = True)(x)
    return x

def up_conv(inputs, filters ,bilinear):
    if bilinear:
        x = tf.keras.layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(inputs)
        x = tf.keras.Conv2D(filters=filters, kernel_size=3, strides=1, 
                            padding='same', use_bias=True)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
    else:
        x = tf.keras.layers.Conv2DTranspose(filters=filters, kernel_size=2, strides=2)(inputs)
    return x


def dualAttention(g, inputs, filters, filters_g, ratio):
    x1 = tf.keras.layers.Conv2D(filters=filters, kernel_size=1, use_bias=True)(inputs)

    avg_out = tfa.layers.AdaptiveAveragePooling2D(output_size=(1,1))(x1)
    avg_out = tf.keras.layers.Conv2D(filters=filters//ratio, kernel_size=1, use_bias=True)(avg_out)
    avg_out = tf.keras.layers.LeakyReLU()(avg_out)
    avg_out = tf.keras.layers.Conv2D(filters=filters, kernel_size=1, use_bias=True)(avg_out)

    max_out = tfa.layers.AdaptiveMaxPooling2D(output_size=(1,1))(x1)
    max_out = tf.keras.layers.Conv2D(filters=filters//ratio, kernel_size=1, use_bias=True)(max_out)
    max_out = tf.keras.layers.LeakyReLU()(max_out)
    max_out = tf.keras.layers.Conv2D(filters=filters, kernel_size=1, use_bias=True)(max_out)
    
    out = avg_out + max_out
    del avg_out, max_out

    x1 = x1 * tf.keras.activations.sigmoid(out)

    g1 = tf.keras.layers.Conv2D(filters=filters_g, kernel_size=1, strides=1, 
                                padding='same', use_bias=True)(g)
    g1 = tf.keras.layers.BatchNormalization()(g1)
    x2 = tf.keras.layers.Conv2D(filters=filters_g, kernel_size=1, strides=1, 
                                padding='same', use_bias=True)(x1)
    x2 = tf.keras.layers.BatchNormalization()(x2)

    leakyReLU = tf.keras.layers.LeakyReLU()(g1 + x2)
    psi = tf.keras.layers.Conv2D(filters=1, kernel_size=1, strides=1, 
                                padding='same', use_bias=True)(leakyReLU)
    psi = tf.keras.layers.BatchNormalization()(psi)
    psi = tf.keras.activations.sigmoid(psi)
    
    # print(filters.shape, psi.shape)
    return inputs * psi
    # return avg_out

def MDA_Net(input_shape=(256,256,3), filters_number= [32, 64, 128, 256, 512], output_ch=1):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x1 = conv_3_1(inputs, filters=filters_number[0])
    
    x2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(x1)
    x2 = conv_3_1(x2, filters=filters_number[1])

    x3 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(x2)
    x3 = conv_3_1(x3, filters=filters_number[2])

    x4 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(x3)
    x4 = conv_3_1(x4, filters=filters_number[3])

    x5 = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(x4)
    x5 = conv_3_1(x5, filters=filters_number[4])

    d5 = up_conv(x5, filters=filters_number[3], bilinear=False)
    x41 = dualAttention(d5, x4, filters=filters_number[3], filters_g=filters_number[2], ratio=2)
    d5 = tf.keras.layers.Concatenate(axis=-1)([x41, d5])
    d5 = conv_3_1(d5, filters=filters_number[3])

    d4 = up_conv(d5, filters=filters_number[2], bilinear=False)
    x31 = dualAttention(d4, x3, filters=filters_number[2], filters_g=filters_number[1], ratio=2)
    d4 = tf.keras.layers.Concatenate(axis=-1)([x31, d4])
    d4 = conv_3_1(d4, filters=filters_number[2])

    d3 = up_conv(d4, filters=filters_number[1], bilinear=False)
    x21 = dualAttention(d3, x2, filters=filters_number[1], filters_g=filters_number[0], ratio=2)
    d3 = tf.keras.layers.Concatenate(axis=-1)([x21, d3])
    d3 = conv_3_1(d3, filters=filters_number[1])

    d2 = up_conv(d3, filters=filters_number[0], bilinear=False)
    x11 = dualAttention(d2, x1, filters=filters_number[0], filters_g=16, ratio=2)
    d2 = tf.keras.layers.Concatenate(axis=-1)([x11, d2])
    d2 = conv_3_1(d2, filters=filters_number[0])

    d1 = tf.keras.layers.Conv2D(filters=output_ch, kernel_size=1, strides=1, 
                                padding='same')(d2)
    d1 = tf.keras.activations.sigmoid(d1)

    model = tf.keras.models.Model(inputs=inputs, outputs=d1)
    return model

model = MDA_Net(input_shape=(256,256,3), filters_number= [32, 64, 128, 256, 512], output_ch=1)
# model.summary()

2022-08-16 08:08:58.937187: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4873 MB memory:  -> device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:05:00.0, compute capability: 6.1


In [3]:
from preprocess.prepare_dataset import data_gen

out_path = '/home/quyet/DATA_ML/Projects/road_thailand/tmp'
overlap_mask = os.path.join(out_path, 'mask_cut_crop')
train_dataset, valid_dataset, _, _ = data_gen(os.path.join(overlap_mask, '*.tif'), img_size=256, 
                                                            batch_size=2, N_CLASSES=1, numband=3, 
                                                            split_ratios=0.8, test_data=False, multi=False)

Training:validation = 924:232


In [4]:
from models import loss
from models.metrics import iou, dice_coef
from models.callback.save_best import SavebestweightsandEarlyStopping

model_name = 'mdanet'
mission = 'road_thailand'
img_size = 256
num_class = 1 
batch_size = 2

def lr_decay(epoch):
    initial_learningrate=1e-3
    if epoch < 1:
        return initial_learningrate
    else:
        return initial_learningrate * 0.9 ** (epoch)

if batch_size >1:
    val_batch_size = int(batch_size/2)
else:
    val_batch_size = batch_size
    
print("Init metric function")
if num_class==1:
    recall = tf.keras.metrics.Recall()
    precision = tf.keras.metrics.Precision()
    model_metrics = [precision, recall, dice_coef, iou, tf.keras.metrics.BinaryAccuracy(threshold=0.5)]
else:
    recall = tf.keras.metrics.Recall()
    precision = tf.keras.metrics.Precision()
    accuracy = tf.keras.metrics.CategoricalAccuracy()
    model_metrics = [precision, recall, dice_coef, iou, accuracy]
    
checkpoint_filepath= '/home/quyet/DATA_ML/Projects/segmentation/logs/tmp'
log_dir = '/home/quyet/DATA_ML/Projects/segmentation/logs/graph'
weights_path = '/home/quyet/DATA_ML/WorkSpace/segmentation/weights/%s/'%(model_name) +model_name+'_'+mission+'_'+str(img_size)+'_'+str(num_class)+'class.h5'
patience = 10
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only= True, 
                                                                monitor='val_loss', mode='min', save_best_only=True)
model_lrscheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=1)
model_lrreduce_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience, min_lr=1e-7, verbose=1)
model_earlystopping_callback = SavebestweightsandEarlyStopping(patience=patience, weights_path=weights_path)
model_endtrainnan_callback = tf.keras.callbacks.TerminateOnNaN()
model_tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)
model_callbacks = [model_checkpoint_callback, model_lrscheduler_callback,
                    model_lrreduce_callback, model_earlystopping_callback,
                    model_tensorboard_callback,]

# model = FF_UNet(attention_gates=True)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer = optimizer, loss = loss.balanced_cross_entropy_loss,
             metrics = model_metrics)

model.load_weights('/home/quyet/DATA_ML/WorkSpace/segmentation/weights/mdanet/mdanet_road_256_1class_train.h5')
history_train = model.fit(train_dataset, batch_size=batch_size, epochs=100, verbose=1, 
                      callbacks=model_callbacks, validation_data=valid_dataset, 
                      validation_batch_size=val_batch_size, use_multiprocessing=True)

Init metric function

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/100


2022-08-16 08:09:22.994881: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2022-08-16 08:09:24.968390: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


    462/Unknown - 452s 929ms/step - loss: 0.0865 - precision: 0.9686 - recall: 0.7530 - dice_coef: 0.8041 - iou: 0.6743 - binary_accuracy: 0.7581
Save best train weights.
Save best val weights.


2022-08-16 08:18:42.021242: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 786432000 exceeds 10% of free system memory.
2022-08-16 08:18:42.704805: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 566231040 exceeds 10% of free system memory.
2022-08-16 08:18:43.035959: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1572864000 exceeds 10% of free system memory.
2022-08-16 08:18:45.233483: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 786432000 exceeds 10% of free system memory.


462/462 [==============================] - 581s 1s/step - loss: 0.0865 - precision: 0.9686 - recall: 0.7530 - dice_coef: 0.8041 - iou: 0.6743 - binary_accuracy: 0.7581 - val_loss: 0.0916 - val_precision: 0.9778 - val_recall: 0.7133 - val_dice_coef: 0.7879 - val_iou: 0.6524 - val_binary_accuracy: 0.7346 - lr: 0.0010

Epoch 00002: LearningRateScheduler setting learning rate to 0.0009000000000000001.
Epoch 2/100
462/462 [==============================] - ETA: 0s - loss: 0.0706 - precision: 0.9761 - recall: 0.8200 - dice_coef: 0.8476 - iou: 0.7366 - binary_accuracy: 0.8226
Save best train weights.
Save best val weights.


2022-08-16 08:27:41.612045: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 786432000 exceeds 10% of free system memory.


462/462 [==============================] - 539s 1s/step - loss: 0.0706 - precision: 0.9761 - recall: 0.8200 - dice_coef: 0.8476 - iou: 0.7366 - binary_accuracy: 0.8226 - val_loss: 0.0865 - val_precision: 0.9503 - val_recall: 0.9418 - val_dice_coef: 0.9006 - val_iou: 0.8206 - val_binary_accuracy: 0.9050 - lr: 9.0000e-04

Epoch 00003: LearningRateScheduler setting learning rate to 0.0008100000000000001.
Epoch 3/100
462/462 [==============================] - ETA: 0s - loss: 0.0649 - precision: 0.9793 - recall: 0.8327 - dice_coef: 0.8601 - iou: 0.7555 - binary_accuracy: 0.8355
Save best train weights.
Save best val weights.
462/462 [==============================] - 576s 1s/step - loss: 0.0649 - precision: 0.9793 - recall: 0.8327 - dice_coef: 0.8601 - iou: 0.7555 - binary_accuracy: 0.8355 - val_loss: 0.0665 - val_precision: 0.9867 - val_recall: 0.8093 - val_dice_coef: 0.8456 - val_iou: 0.7340 - val_binary_accuracy: 0.8223 - lr: 8.1000e-04

Epoch 00004: LearningRateScheduler setting learnin

In [7]:
import cv2
import numpy as np

from tqdm import tqdm
from osgeo import gdal
from postprocess.convert_tif import dilation_obj, remove_small_items, write_image

def get_im_by_coord(org_im, start_x, start_y,num_band, padding, crop_size, input_size):
    startx = start_x-padding
    endx = start_x+crop_size+padding
    starty = start_y - padding
    endy = start_y+crop_size+padding
    result=[]
    img = org_im[starty:endy, startx:endx]
    img = img.swapaxes(2,1).swapaxes(1,0)
    for chan_i in range(num_band):
        result.append(cv2.resize(img[chan_i],(input_size, input_size), interpolation = cv2.INTER_CUBIC))
    return np.array(result).swapaxes(0,1).swapaxes(1,2)

def get_img_coords(w, h, padding, crop_size):
    new_w = w + 2*padding
    new_h = h + 2*padding
    cut_w = list(range(padding, new_w - padding, crop_size))
    cut_h = list(range(padding, new_h - padding, crop_size))

    list_hight = []
    list_weight = []
    for i in cut_h:
        if i < new_h - padding - crop_size:
            list_hight.append(i)
    list_hight.append(new_h-crop_size-padding)

    for i in cut_w:
        if i < new_w - crop_size - padding:
            list_weight.append(i)
    list_weight.append(new_w-crop_size-padding)

    img_coords = []
    for i in list_weight:
        for j in list_hight:
            img_coords.append([i, j])
    return img_coords

def padded_for_org_img(values, num_band, padding):
    padded_org_im = []
    for i in range(num_band):
        band = np.pad(values[i], padding, mode='reflect')
        padded_org_im.append(band)

    values = np.array(padded_org_im).swapaxes(0,1).swapaxes(1,2)
    print(values.shape)
    del padded_org_im
    return values

def predict(model, values, img_coords, num_band, h, w, padding, crop_size, 
            input_size, batch_size, thresh_hold, choose_stage):
    cut_imgs = []
    for i in range(len(img_coords)):
        im = get_im_by_coord(values, img_coords[i][0], img_coords[i][1],
                            num_band,padding, crop_size, input_size)
        cut_imgs.append(im)

    a = list(range(0, len(cut_imgs), batch_size))

    if a[len(a)-1] != len(cut_imgs):
        a[len(a)-1] = len(cut_imgs)

    y_pred = []
    for i in tqdm(range(len(a)-1)):
        x_batch = []
        x_batch = np.array(cut_imgs[a[i]:a[i+1]])
        # print(x_batch.shape)
        img_edge = []
        # for img_x in x_batch:
        #     lab_batch = color.rgb2lab(img_x)  
            # img_edge.append(cv2.Canny(np.asarray(np.uint8(lab_batch)),0,0)[..., np.newaxis])
        # print(img_edge.shape)
        # img_edge = np.array(img_edge)
        
        # print(x_batch.shape, img_edge.shape)
        # y_batch = model.predict((x_batch/255, img_edge/255))
        y_batch = model.predict(x_batch/255)
        if len(model.outputs)>1:
            y_batch = y_batch[choose_stage]
        mutilabel = False
        if y_batch.shape[-1]>=2:
            mutilabel = True
            y_batch = np.argmax(y_batch, axis=-1)
        # print(np.unique(y_batch), y_batch.shape)
            
        y_pred.extend(y_batch)
    big_mask = np.zeros((h, w)).astype(np.float16)
    for i in range(len(cut_imgs)):
        true_mask = y_pred[i].reshape((input_size,input_size))
        if not mutilabel:
            true_mask = (true_mask>thresh_hold).astype(np.uint8)
            true_mask = (cv2.resize(true_mask,(input_size, input_size), interpolation = cv2.INTER_CUBIC)>thresh_hold).astype(np.uint8)
            # true_mask = true_mask.astype(np.float16)
        start_x = img_coords[i][1]
        start_y = img_coords[i][0]
        big_mask[start_x-padding:start_x-padding+crop_size, start_y-padding:start_y -
                    padding+crop_size] = true_mask[padding:padding+crop_size, padding:padding+crop_size]
    del cut_imgs
    return big_mask

img_size = 256
num_band = 3
crop_size = 200
batch_size = 1
thresh_hold = 0.8
choose_stage = 0

model.load_weights('/home/quyet/DATA_ML/WorkSpace/segmentation/weights/mdanet/mdanet_road_256_1class_train.h5')
image_path = '/home/quyet/DATA_ML/Projects/road_multi/crop/img/test.tif'
dataset = gdal.Open(image_path)
values = dataset.ReadAsArray()[0:num_band]
h,w = values.shape[1:3]    
padding = int((img_size - crop_size)/2)
img_coords = get_img_coords(w, h, padding, crop_size)
values = padded_for_org_img(values, num_band, padding)
big_mask = predict(model, values, img_coords, num_band, h, w, padding, crop_size, 
                    img_size, batch_size, thresh_hold, choose_stage)

(6237, 6126, 3)


100%|█████████████████████████████████████████| 960/960 [02:10<00:00,  7.37it/s]


In [8]:
image_path = '/home/quyet/DATA_ML/Projects/road_multi/crop/img/test.tif'
result_path = write_image(image_path, big_mask)

Write image...
